In [ ]:
import pandas as pd
import regex as re

In [2]:
df=pd.read_csv('./sz_nongshanghang_20200610.csv')
msg=df['Message'].to_list()
msg=list(set(msg))

ParserError: Error tokenizing data. C error: Expected 6 fields in line 152, saw 7


In [ ]:
split_pt=re.compile(r'([\s\(\)\[\]\'\"=,;。，（）【】]|[\u4e00-\u9fa5])')

In [ ]:
msg_tokens=[]
for m in msg:
    tokens=[token for token in split_pt.split(m) if token!='' and token!=None]
    msg_tokens.append(tokens)

for tokens in msg_tokens:
    for i,token in enumerate(tokens):
        tokens[i:i+1]=[t for t in re.split(r'([:：]$|(?<=(?:IP)|^)[:：])',token) if t!='']

pd.DataFrame({'msg_tokens':msg_tokens}).to_csv('tokens.csv')

In [ ]:
re_date=r'\d{4}-\d{2}-\d{2}'
re_time=r'^\d{2}:\d{2}:\d{2}$'

re_cls=r'(\.([A-Z]+[a-z]*)+)|([A-Z]+[a-z]+){2,}$'
re_path=r'^\'?\/[a-zA-Z0-9]'
re_ip=r'^(?:[0-9]{1,3}\.){3}[0-9]{1,3}'

re_url=r'[a-z]+:\/\/\S*'

In [ ]:
def sub_tag(ntokens,ntags, patt_tag_list):

    def _sub_tag(ntokens,ntags, patt, target_tag):
        for tokens,tags in zip(ntokens,ntags):
            for i in range(len(tokens)):
                if re.search(patt,tokens[i]):
                    tags[i]=target_tag

    for patt,tag in patt_tag_list:
        _sub_tag(ntokens,ntags,patt,tag)

In [ ]:
msg_anno=[]
for tokens in msg_tokens:
    anno=['O']*len(tokens)
    msg_anno.append(anno)

sub_tag(msg_tokens,msg_anno,
    [(re_date,'B-DATE'),
    (re_time,'B-TIME'),
    (re_cls,'B-CLS'),
    (re_path,'B-PATH'),
    (re_ip,'B-IP'),
    (re_url,'B-URL')])

In [ ]:
with open('./msg_check.txt','w') as f:
    for tokens,tags in zip(msg_tokens,msg_anno):
        for token,tag in zip(tokens,tags):
            f.write(token+' '+tag+'\n')
        f.write('\n')

In [ ]:
df = pd.DataFrame({
    'tokens': msg_tokens,
    'tags': msg_anno,
    'raw_log': msg
})
df.to_csv('./nongshanghang_ds.csv',)


In [ ]:
df.loc[415:425]

,tokens,tags,raw_log
415,"[蚂, 蚁, 金, 融, 科, 技, 监, 控, ,, 您, 的, ACS, （, dr-i...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","蚂蚁金融科技监控,您的ACS（dr-ibs-edeposit-g-1-1）在租户(金融云租户..."
416,"[[, ], 上, 传, 龙, 岗, 国, 库, 文, 件, [, 000-305-2020...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",[]上传龙岗国库文件[000-305-2020052809184682210623.xml]失败
417,"[蚂, 蚁, 金, 融, 科, 技, 核, 心, 态, 监, 控, antstackants...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","蚂蚁金融科技核心态监控antstackantstack,您的ACS（/10.168.80.8..."
418,"[行, 内, 微, 信, 通, 知, 发, 送, 出, 现, 异, 常, ，, 10, 分,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","行内微信通知发送出现异常，10分钟内发送异常数目超过1500,异常数目为2231，请及时检查..."
419,"[200061, 日, 启, 批, 量, 入, 账, :, 200061, 联, 机, 账,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",200061日启批量入账:200061联机账务处理结果处理失败【0】笔\r\n处理成功【12...
420,"[蚂, 蚁, 金, 融, 科, 技, 核, 心, 态, 监, 控, antstackants...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","蚂蚁金融科技核心态监控antstackantstack,您的ACS（/192.168.80...."
421,"[新, 手, 机, 银, 行, 告, 警, ,, 时, 间, 段, 为, [, 2020-0...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-DATE...","新手机银行告警,时间段为[2020-06-01 22:55:55]至[2020-06-01 ..."
422,"[蚂, 蚁, 金, 融, 科, 技, 监, 控, ,, 您, 的, ACS, （, prd-...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","蚂蚁金融科技监控,您的ACS（prd-ibs-socialinsurance-1-2）在租户..."
423,"[蚂, 蚁, 金, 融, 科, 技, 监, 控, ,, 您, 的, ACS, （, dr-i...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","蚂蚁金融科技监控,您的ACS（dr-ibs-eesbapi-g-1-1）在租户(金融云租户)..."
424,"[[, NSTER01, ], 2019-12-03, , 16:06:50, 时, ，,...","[O, O, O, B-DATE, O, B-TIME, O, O, O, O, O, O,...",[NSTER01]2019-12-03 16:06:50时，dx5服务器(192.0.27....
